<a href="https://colab.research.google.com/github/HarshaVardhanLanka/OpenDeepResearchAgent/blob/main/Day_7(Model_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai tavily-python pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 5.3 MB/s eta 0:00:00


In [2]:
import os
import io
import json
import datetime
import google.generativeai as genai
from tavily import TavilyClient
from google.colab import userdata
from google.colab import files
import pypdf
from IPython.display import display, Markdown

# --- API CONFIGURATION ---
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    TAVILY_API_KEY = userdata.get('TAVILY')
except:
    print("⚠️ Error: Please set GOOGLE_API_KEY and TAVILY_API_KEY in Colab Secrets.")

genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro-latest')
tavily = TavilyClient(api_key=TAVILY_API_KEY)
print("Setup complete.")

Setup complete.


In [5]:
# --- MEMORY SYSTEM ---
class HistoryManager:
    def __init__(self):
        self.history_file = "agent_history.json"
        self.history = self.load_history()

    def load_history(self):
        """Loads history from a JSON file if it exists."""
        if os.path.exists(self.history_file):
            with open(self.history_file, 'r') as f:
                return json.load(f)
        return []

    def save_entry(self, input_text, mode, final_report):
        """Saves a new research session to memory."""
        entry = {
            "id": len(self.history) + 1,
            "timestamp": str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")),
            "mode": mode,
            "input": input_text[:15] + "..." if len(input_text) > 50 else input_text, # Preview only
            "full_input": input_text,
            "report": final_report
        }
        self.history.append(entry)
        with open(self.history_file, 'w') as f:
            json.dump(self.history, f)
        print("✅ Research saved to History.")

    def show_history(self):
        """Displays list of past inputs."""
        if not self.history:
            print("\n📭 History is empty.")
            return None

        print("\n📚 RESEARCH HISTORY:")
        print(f"{'ID':<4} | {'Time':<18} | {'Mode':<8} | {'Topic/Input'}")
        print("-" * 60)
        for item in self.history:
            print(f"{item['id']:<4} | {item['timestamp']:<18} | {item['mode']:<8} | {item['input']}")
        return self.history
# Initialize Memory
memory = HistoryManager()

# --- HELPER: PDF TEXT EXTRACTOR ---
def extract_pdf_text(uploaded_file_bytes):
    try:
        pdf_reader = pypdf.PdfReader(io.BytesIO(uploaded_file_bytes))
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Error reading PDF: {e}"

# --- STRICT AGENT STEPS ---

def step_1_generate_questions(input_text, mode):
    """Generates 3 questions based on mode."""
    print(f"\nStep 1: Generating 3 sub-questions (Mode: {mode})...")

    if mode == "topic":
        prompt = f"""
        Task: Break down the topic "{input_text}" into exactly 3 searchable questions.
        Constraint: Output ONLY the 3 questions separated by newlines. No numbering.
        """
    else: # mode == pdf
        prompt = f"""
        Task: Read the following paper excerpt and generate 3 search questions to VALIDATE the claims or check for newer updates.
        Paper Context: {input_text[:10000]}
        Constraint: Output ONLY the 3 questions separated by newlines. No numbering.
        """

    response = model.generate_content(prompt)
    questions = [q.strip() for q in response.text.strip().split('\n') if q.strip()]
    return questions[:3]

def step_2_get_responses(questions):
    """Retrieves short responses (Max 1 result per question)."""
    print(f"Step 2: Retrieving short responses from the web...")
    q_and_a_results = []

    for i, q in enumerate(questions, 1):
        print(f"   {i}. Searching: {q}")
        try:
            # max_results=1 forces a short response
            search_result = tavily.search(query=q, search_depth="basic", max_results=1)

            if search_result['results']:
                content = search_result['results'][0]['content'][:600] # Limit char count
                q_and_a_results.append(f"Question: {q}\nFound Info: {content}")
            else:
                q_and_a_results.append(f"Question: {q}\nFound Info: No data.")
        except:
             q_and_a_results.append(f"Question: {q}\nFound Info: Search Failed.")

    return "\n---\n".join(q_and_a_results)

def step_3_summarize(input_text, research_data, mode="topic"):
    """Summarizes everything into a final report."""
    print(f"📝 Synthesizing final report...")

    if mode == "topic":
        prompt = f"""
        Write a comprehensive research summary on the topic below, incorporating the search findings.

        User Topic: {input_text}
        Search Data: {research_data}

        Format: Markdown with headers.
        """
    else:
        prompt = f"""
        Create a "Deep Dive" analysis of the provided research paper using the external search data.

        1. Summarize the Paper's core contribution.
        2. Compare it with the "External Validation" findings (State-of-the-art, updates, or contradictions).
        3. Provide a final conclusion on the paper's relevance today.

        Paper Text (Context): {input_text[:15000]}
        External Validation Data: {research_data}

        Format: Markdown with headers.
        """

    response = model.generate_content(prompt)

    if not response.candidates:
        feedback = response.prompt_feedback
        error_message = f"Model did not return any candidates. Finish reason: {feedback.finish_reason}. Safety ratings: {feedback.safety_ratings}"
        print(f"Error: {error_message}")
        return f"**Error generating report:** {error_message}"
    try:
        return response.text
    except ValueError as e:
        feedback = response.prompt_feedback
        error_message = f"Failed to access response.text: {e}. Finish reason: {feedback.finish_reason}. Safety ratings: {feedback.safety_ratings}"
        print(f"Error: {error_message}")
        return f"**Error generating report:** {error_message}"

In [7]:
# --- MAIN LOOP ---

def main():
    while True:
        print("\n" + "="*15 + " STRICT HYBRID AGENT " + "="*15)
        print("1. Research a Topic (Text 💬)")
        print("2. Research a Paper (Upload PDF 📖)")
        print("3. View History 📜")
        print("4. Exit 📤")

        choice = input("Select Option: ")

        if choice == '4':
            print("Goodbye.👋")
            break

        # --- OPTION 3: HISTORY ---
        if choice == '3':
            history_data = memory.show_history()
            if history_data:
                sub_choice = input("\nEnter ID to view full report (or Press Enter to go back): ")
                if sub_choice.isdigit():
                    idx = int(sub_choice) - 1
                    if 0 <= idx < len(history_data):
                        entry = history_data[idx]
                        print(f"\n🔁 RECALLING REPORT FOR: {entry['input']}")
                        print("-" * 50)
                        display(Markdown(entry['report']))
                        input("\nPress Enter to continue...")
            continue # Skip the rest and go back to menu

        # SETUP INPUT
        mode = ""
        input_data = ""
        label = "" # Used for history title

        if choice == '1':
            mode = "topic"
            input_data = input("\nEnter Research Topic: ")
            label = input_data

        elif choice == '2':
            mode = "pdf"
            print("\n⬆️ Upload PDF now:")
            uploaded = files.upload()
            if uploaded:
                filename = next(iter(uploaded))
                input_data = extract_pdf_text(uploaded[filename])
                label = f"PDF: {filename}"
            else:
                print("No file uploaded.")
                continue

        if input_data:
            # --- EXECUTE 3-STEP PROCESS ---

            # 1. Generate Questions
            questions = step_1_generate_questions(input_data, mode)
            print(f"   -> Plan: {questions}")

            # 2. Retrieve Responses
            raw_data = step_2_get_responses(questions)

            # 3. Summarize
            final_report = step_3_summarize(input_data, raw_data, mode)

            print("\n" + "="*20 + " 🎯 FINAL REPORT: " + "="*20 + "\n")
            display(Markdown(final_report))

            # Save to History
            memory.save_entry(label, mode, final_report)

if __name__ == "__main__":
    main()


=============== STRICT HYBRID AGENT ===============
1. Research a Topic (Text 💬)
2. Research a Paper (Upload PDF 📖)
3. View History 📜
4. Exit 📤
Select Option: 3

📚 RESEARCH HISTORY:
ID   | Time               | Mode     | Topic/Input
------------------------------------------------------------
1    | 2025-11-26 15:03   | pdf      | PDF: 2511.17546v1.pdf

Enter ID to view full report (or Press Enter to go back): 1

🔁 RECALLING REPORT FOR: PDF: 2511.17546v1.pdf
--------------------------------------------------


Here is a "Deep Dive" analysis of the research paper by Rathi et al. based on the provided text and external search data.

***

### Deep Dive: Reference Quadrupole Moments from Muonic Atom Lamb Shifts

#### 1. Paper's Core Contribution

The paper by Rathi et al. proposes a novel experimental method to precisely measure the absolute electric quadrupole moments (Q) for a specific group of elements: the light transition metals (from Vanadium, Z=23 to Zinc, Z=30).

**The Problem:** The accuracy of Q values for all isotopes of an element is limited by the precision of a single "reference" value, `Q_ref`, typically measured for a stable isotope. For transition metals, determining `Q_ref` is challenging. The standard method involves atomic spectroscopy, which requires highly accurate theoretical calculations of the Electric Field Gradient (EFG) at the nucleus. These calculations are notoriously difficult and uncertain for atoms with open d-shell electronic configurations, leading to large uncertainties in `Q_ref` for this region of the periodic table.

**The Proposed Solution:** The authors advocate for using **muonic atoms** to bypass the problem of complex electronic structure calculations. In a muonic atom, a muon orbits the nucleus. Because a muon is ~200 times heavier than an electron, it orbits much closer to the nucleus, offering three key advantages:
1.  **Massive EFG:** The EFG created by the muon is enormous (~10⁷ times larger than from an electron), leading to large, measurable energy splittings.
2.  **Simple Theory:** The EFG can be calculated with very high precision (<1%) because the muonic atom is a simple, hydrogen-like system, largely unaffected by the remaining atomic electrons.
3.  **Clean Signal:** The quadrupole interaction dominates over the magnetic dipole interaction, simplifying the analysis.

**The Innovation:** While using muonic atoms is not new, previous experiments were limited by detector technology. The authors propose measuring the weakly populated but highly sensitive **2s-2p Lamb shift transition** (in the 10-40 keV X-ray range) using **cryogenic microcalorimeters (MCs)**. These detectors provide the crucial combination of high energy resolution (~10 eV) and high efficiency necessary to resolve the hyperfine splitting of these faint transitions.

Through detailed simulations for `⁶³Cu`, the paper establishes the feasibility of this approach, demonstrating that it could improve the accuracy of `Q_ref` by up to an order of magnitude within a single day of measurement time at a facility like the Paul Scherrer Institute.

#### 2. Comparison with External Validation Findings

This section compares the paper's claims and methods with the provided external search data, highlighting state-of-the-art updates and potential contradictions.

**State-of-the-Art and Updates:**

*   **Detector Technology (Validation):** The paper's entire premise relies on the performance of cryogenic microcalorimeters. The external data snippet mentioning the "Development of cryogenic X-ray detectors based on Mo/Au TES" (Transition-Edge Sensors) *strongly validates* the paper's technological foundation. It confirms that this detector technology is an active area of advanced research, with ongoing efforts to "enable further enhancement of the achieved energy resolution." This suggests that the experimental conditions proposed by Rathi et al. are not only feasible but likely to improve, strengthening the paper's argument.

*   **Competing Methodology (Update):** The paper argues that its muonic method is necessary because theoretical EFG calculations for transition metals are unreliable. However, the external data on `⁶³Cu` presents a significant update to this landscape. The finding that the `⁶³Cu` quadrupole moment was recently determined by combining experimental constants with "relativistic Dirac-Coulomb coupled cluster calculations of the electric field gradient (EFG)" shows that the *ab initio* theoretical approach has made substantial progress. This sophisticated computational method directly tackles the electronic correlation problem that the paper identifies as a major roadblock. This means the muonic atom method is no longer the *only* path forward; it is now in competition with a rapidly advancing theoretical front.

**Contradictions:**

*   **Premise of Necessity:** There is no direct contradiction of the physics presented in the paper. The advantages of muonic atoms are well-established. The "contradiction" is in the paper's underlying premise of *necessity*. The authors frame their method as a way to overcome a challenge that is "poorly understood" in conventional atomic systems. The external data on `⁶³Cu` suggests that this challenge, while difficult, is now being successfully met by cutting-edge quantum chemistry. Therefore, the muonic method's role may be shifting from a unique solution to a complementary benchmark.

*   **Irrelevant Data:** The external data on "Experimental Lamb-dip spectra" in H₂O is unrelated. It refers to a different physical phenomenon (molecular rovibrational transitions) and a different experimental technique (NICE-OHMS). It does not validate or contradict the paper's proposal concerning muonic atoms.

#### 3. Final Conclusion on the Paper's Relevance Today

The paper by Rathi et al. remains highly relevant, though its role in the scientific landscape has evolved in light of recent progress in competing fields.

1.  **Gold Standard Potential:** The proposed method is fundamentally robust. A measurement based on a simple, hydrogen-like muonic system is less susceptible to theoretical modeling errors than one relying on complex many-body electronic structure calculations. It therefore stands as a potential **"gold standard"** for determining `Q_ref`, providing a clean, experiment-driven value that is largely independent of atomic theory.

2.  **Technological Timeliness:** The technological cornerstone of the proposal—cryogenic microcalorimetry—is more advanced today than at the time of writing, making the proposed experiment even more feasible.

3.  **Crucial Benchmark:** Given the parallel advances in theoretical EFG calculations, the paper's proposed measurement has acquired a new and critical purpose: **to serve as a definitive benchmark.** If the muonic atom measurement of `Q_ref` for an element like `⁶³Cu` agrees with the value derived from the latest atomic coupled-cluster calculations, it would provide a powerful, cross-disciplinary validation of our understanding of relativistic quantum chemistry. If the values disagree, it would point to subtle, yet-unaccounted-for effects in either nuclear or atomic physics, stimulating new theoretical developments.

In conclusion, the paper outlines an elegant and powerful experimental approach. While no longer the only viable path to high-precision quadrupole moments for transition metals, it is arguably the cleanest. Its relevance today is as a crucial, first-principles benchmark that can be used to validate, challenge, and ultimately advance the state-of-the-art in both nuclear structure physics and theoretical atomic chemistry.


Press Enter to continue...

=============== STRICT HYBRID AGENT ===============
1. Research a Topic (Text 💬)
2. Research a Paper (Upload PDF 📖)
3. View History 📜
4. Exit 📤
Select Option: 2

⬆️ Upload PDF now:


Saving 2511.17546v1.pdf to 2511.17546v1 (1).pdf

Step 1: Generating 3 sub-questions (Mode: pdf)...
   -> Plan: ['Recent advancements in cryogenic microcalorimeter performance for muonic x-ray spectroscopy in the 10-40 keV range.', 'Experimental realization of muonic Lamb shift measurements to determine quadrupole moments in transition elements.', 'Updated reference values for electric quadrupole moments of transition metals (Z=23-30) since 2016.']
Step 2: Retrieving short responses from the web...
   1. Searching: Recent advancements in cryogenic microcalorimeter performance for muonic x-ray spectroscopy in the 10-40 keV range.
   2. Searching: Experimental realization of muonic Lamb shift measurements to determine quadrupole moments in transition elements.
   3. Searching: Updated reference values for electric quadrupole moments of transition metals (Z=23-30) since 2016.
📝 Synthesizing final report...

==================== 🎯 FINAL REPORT: ====================



### **Deep Dive Analysis: Reference Quadrupole Moments from Muonic Lamb Shifts**

This analysis examines the research paper by S. Rathi et al., which proposes a novel method for measuring nuclear electric quadrupole moments in transition elements, and evaluates its standing against external data.

---

#### **1. Paper's Core Contribution**

The core contribution of this paper is a **detailed feasibility study for a novel experimental method designed to significantly improve the accuracy of reference nuclear electric quadrupole moments (Q_ref) for light transition elements (Z=23 to Z=30).**

The authors identify a critical gap in nuclear data: the Q_ref values for elements like vanadium, chromium, manganese, and copper are poorly known. This is because traditional methods, which rely on atomic or molecular spectroscopy, are hampered by the immense difficulty of accurately calculating the electric field gradient (EFG) at the nucleus for elements with complex, open *d*-shell electronic configurations. Since the uncertainty in a single Q_ref propagates across the entire isotopic chain, this is a major limiting factor for nuclear structure studies.

The proposed solution is a synergy of two key components:

1.  **Muonic Atoms:** By replacing electrons with muons, a hydrogen-like system is created. The muon's heavy mass (~207x the electron's) results in an orbit much closer to the nucleus. This has two profound advantages:
    *   The EFG is enhanced by a factor of ~10⁷, leading to large, easily measurable hyperfine splittings.
    *   The system is theoretically "clean." The EFG can be calculated with high precision, bypassing the complex electron correlation problems that plague normal atomic calculations.

2.  **Cryogenic Microcalorimeters (MCs):** Previous attempts to use muonic atoms for this purpose were stalled by detector limitations. The crucial muonic transitions (specifically the 2s-2p Lamb shift) are weak and produce x-rays in the 10-40 keV range. Conventional solid-state detectors lack the resolution to distinguish the hyperfine structure, while high-resolution crystal spectrometers have extremely low efficiency, making the measurement of such weak signals impractical. The authors propose using MCs, which uniquely offer both **superb energy resolution (~10 eV)** and **high quantum efficiency** in this energy regime.

The paper's contribution is not an experimental result but a comprehensive theoretical and simulation-based argument (using MCDFGME and GEANT4) demonstrating that this experiment is viable. They calculate the expected signals, optimize experimental parameters like muon beam momentum, and show that an order-of-magnitude improvement in the accuracy of Q_ref is achievable within a reasonable measurement time (e.g., one day).

---

#### **2. Comparison with External Validation Findings**

The external search data provides strong validation for the paper's premises and highlights its continued relevance.

*   **State-of-the-Art Confirmation:** The finding that *"precision x-ray spectroscopy of exotic atoms is currently undergoing a paradigm shift due to the introduction of cryogenic microcalorimeter detectors"* directly confirms the paper's central technological thesis. This external statement validates the authors' choice of MCs as a game-changing technology that makes their proposed measurement possible. It positions the paper not as a speculative idea but as one that leverages a cutting-edge, transformative tool in the field.

*   **No Contradictions or Obsolescence:** The external search for updated quadrupole moments for the target elements (Z=23-30) did not return any new high-precision experimental values. The result for Praseodymium (Pr, Z=59) highlights that progress is being made elsewhere, but the specific "problem region" identified by Rathi et al. appears to remain unsolved. This **lack of new data strongly reinforces the paper's motivation.** It implies that no other competing technique has successfully addressed this challenge in the interim, making the proposed muonic atom experiment as necessary today as when it was proposed.

*   **Methodological Soundness:** The mention of the MCDFGME program aligns perfectly with the methods detailed in the paper. This shows that the authors are using standard, state-of-the-art theoretical codes for their calculations, grounding their feasibility study in a robust computational framework recognized by the field.

In summary, the external data provides no contradictions. Instead, it confirms that the technological foundation of the proposal is sound and that the scientific problem it aims to solve remains a significant, open question in nuclear physics.

---

#### **3. Final Conclusion on the Paper's Relevance Today**

The paper by Rathi et al. is **highly relevant and forward-looking.** It acts as a crucial blueprint for a next-generation experiment poised to resolve a long-standing uncertainty in fundamental nuclear physics.

1.  **Addresses a Persistent Problem:** The inaccuracies in Q_ref for transition metals are a known bottleneck that limits the interpretation of data from radioactive ion beam facilities and hinders the development of nuclear theory in this mass region. The paper offers a clear and viable path to breaking this deadlock.

2.  **Proposes a Realistic Solution:** The relevance of the paper is amplified by the maturation of the proposed technology. As confirmed by the external data, cryogenic microcalorimeters are no longer a developmental curiosity but a paradigm-shifting tool. This makes the proposed experiment more feasible and timely than ever before.

3.  **Impact Beyond Nuclear Physics:** Precise Q_ref values serve as critical benchmarks for state-of-the-art quantum chemistry calculations. By providing an unambiguous experimental value for the nuclear moment, this work would allow theorists to disentangle nuclear effects from electronic effects and rigorously test their complex computational models for open-shell systems.

In conclusion, this paper is not a historical artifact but a **well-defined and compelling research proposal.** Its value lies in its clear identification of a problem, its elegant combination of established physics (muonic atoms) with advanced technology (MCs), and its rigorous demonstration of feasibility. The work serves as a foundational document that will likely guide and justify future experimental efforts at facilities like the Paul Scherrer Institute (PSI). Its relevance will remain high until the proposed experiment is performed and the data gap it identifies is finally closed.

✅ Research saved to History.

=============== STRICT HYBRID AGENT ===============
1. Research a Topic (Text 💬)
2. Research a Paper (Upload PDF 📖)
3. View History 📜
4. Exit 📤
Select Option: 4
Goodbye.👋
